In [1]:
import tensorflow as tf
from tensorflow import keras
import cv2
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import os
import time
from PIL import Image
import matplotlib.pyplot as plt

In [15]:
categories = ['Glioma' , 'healthy' , 'Meningioma' , 'Pituitary tumor']

In [16]:
train_path = r'C:\Projects\Brain Tumor Classifier\brain tumor MRI image_dataset\train'
val_path = r'C:\Projects\Brain Tumor Classifier\brain tumor MRI image_dataset\val'

In [25]:
for category in categories:
    os.mkdir(os.path.join(val_path , category))
    

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Projects\\Brain Tumor Classifier\\brain tumor MRI image_dataset\\val\\Glioma'

In [ ]:
image = cv2.imread(r'C:\Projects\Brain Tumor Classifier\brain tumor MRI image_dataset\train\Glioma\3.png')
print(image.shape)
cv2.imshow('img',image)
plt.show()

(512, 512, 3)


In [ ]:
for category in categories:
    img_count=0
    for img in os.listdir(os.path.join(train_path , category))[0:120]:
        image = Image.open(os.path.join(train_path , category , img))
        image.convert('RGB').save(os.path.join(val_path , category, img) , 'PNG')
        os.remove(os.path.join(train_path , category , img))
        

In [13]:
datagen = ImageDataGenerator(
    
    rescale=1/255.0
)

In [17]:
train_generator = datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse'


)

Found 22486 images belonging to 4 classes.


In [ ]:
print(train_generator.class_indices)

{'Glioma': 0, 'Meningioma': 1, 'Pituitary tumor': 2, 'healthy': 3}


In [18]:
validation_generator = datagen.flow_from_directory(
    val_path,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse'


)

Found 1440 images belonging to 4 classes.


In [51]:
from keras.applications import MobileNet

In [52]:
base_model = MobileNet(weights='imagenet' , include_top=False , input_shape=(224,224,3))

In [53]:
base_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [59]:
for layer in base_model.layers[1:6]:
    layer.trainable = False

for layer in base_model.layers[6:]:
    layer.trainable=True

In [ ]:
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [55]:
model = keras.Sequential([])
model.add(base_model)
model.add(keras.layers.Flatten())
# model.add(keras.layers.Dense(256 , activation='relu'))
model.add(keras.layers.Dense(128 , activation='relu'))
model.add(keras.layers.Dense(4 , activation='softmax'))

In [39]:
model.layers[0].trainable=False


# for layer in model.layers[30:]:
#     layer.trainable=True

In [67]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 50176)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               6422656   
                                                                 
 dense_13 (Dense)            (None, 4)                 516       
                                                                 
Total params: 9,652,036
Trainable params: 9,628,868
Non-trainable params: 23,168
_________________________________________________________________


In [57]:
model.compile(loss = 'sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [28]:
check_point = ModelCheckpoint(
    filepath = 'best_model.s2_v2',
    monitor = 'val_accuracy',
    save_best_only=True,
    save_weights_only=False,
    mode='max',
    verbose=1
)

In [30]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=2,
    mode='max',
    verbose=1
)

In [66]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    epochs=5,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    callbacks=[early_stopping,check_point]
    
)

Epoch 1/5
702/702 [==============================] - ETA: 0s - loss: 0.0612 - accuracy: 0.9787
Epoch 1: val_accuracy improved from 0.95625 to 0.95833, saving model to best_model.v2


INFO:tensorflow:Assets written to: best_model.v2\assets


INFO:tensorflow:Assets written to: best_model.v2\assets


702/702 [==============================] - 1805s 3s/step - loss: 0.0612 - accuracy: 0.9787 - val_loss: 0.1408 - val_accuracy: 0.9583
Epoch 2/5
702/702 [==============================] - ETA: 0s - loss: 0.0563 - accuracy: 0.9813
Epoch 2: val_accuracy did not improve from 0.95833
702/702 [==============================] - 1769s 3s/step - loss: 0.0563 - accuracy: 0.9813 - val_loss: 0.1691 - val_accuracy: 0.9438
Epoch 2: early stopping


In [2]:
load_model = keras.models.load_model('best_model.v2')

In [3]:
load_model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 flatten_5 (Flatten)         (None, 50176)             0         
                                                                 
 dense_12 (Dense)            (None, 128)               6422656   
                                                                 
 dense_13 (Dense)            (None, 4)                 516       
                                                                 
Total params: 9,652,036
Trainable params: 9,626,692
Non-trainable params: 25,344
_________________________________________________________________


In [8]:
load_model.layers[0].trainable = True


In [26]:
load_model.compile(loss = 'sparse_categorical_crossentropy' , optimizer='adam' , metrics=['accuracy'])

In [19]:
load_model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples//train_generator.batch_size,
    epochs=5,
    validation_data = validation_generator,
    validation_steps=validation_generator.samples//validation_generator.batch_size,
    callbacks=[early_stopping,check_point]
    
)

Epoch 1/5
702/702 [==============================] - ETA: 0s - loss: 0.0923 - accuracy: 0.9717
Epoch 1: val_accuracy improved from -inf to 0.96111, saving model to best_model.v2


INFO:tensorflow:Assets written to: best_model.v2\assets


INFO:tensorflow:Assets written to: best_model.v2\assets


702/702 [==============================] - 2565s 4s/step - loss: 0.0923 - accuracy: 0.9717 - val_loss: 0.2713 - val_accuracy: 0.9611
Epoch 2/5
702/702 [==============================] - ETA: 0s - loss: 0.0784 - accuracy: 0.9764
Epoch 2: val_accuracy improved from 0.96111 to 0.97569, saving model to best_model.v2


INFO:tensorflow:Assets written to: best_model.v2\assets


INFO:tensorflow:Assets written to: best_model.v2\assets


702/702 [==============================] - 2925s 4s/step - loss: 0.0784 - accuracy: 0.9764 - val_loss: 0.0858 - val_accuracy: 0.9757
Epoch 3/5
702/702 [==============================] - ETA: 0s - loss: 0.0611 - accuracy: 0.9820
Epoch 3: val_accuracy did not improve from 0.97569
702/702 [==============================] - 1750s 2s/step - loss: 0.0611 - accuracy: 0.9820 - val_loss: 0.4653 - val_accuracy: 0.8778
Epoch 3: early stopping


In [23]:
train_s2_path = r"C:\Projects\Brain Tumor Classifier\brain tumor MRI image_dataset\train_s2"
val_s2_path = r"C:\Projects\Brain Tumor Classifier\brain tumor MRI image_dataset\val_s2"

In [21]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode = 'reflect'
    
    

)

In [35]:
train_s2_generator = datagen.flow_from_directory(
    train_s2_path,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse'


)

Found 5712 images belonging to 4 classes.


In [36]:
print(train_s2_generator.class_indices)

{'Glioma': 0, 'Meningioma': 1, 'Pituitary tumor': 2, 'healthy': 3}


In [25]:
val_s2_generator = datagen.flow_from_directory(
    val_s2_path,
    target_size = (224,224),
    batch_size=32,
    color_mode='rgb',
    class_mode = 'sparse'


)

Found 1311 images belonging to 4 classes.


In [31]:
load_model.fit(
    train_s2_generator,
    steps_per_epoch=train_s2_generator.samples//train_s2_generator.batch_size,
    epochs=4,
    validation_data = val_s2_generator,
    validation_steps=val_s2_generator.samples//val_s2_generator.batch_size,
    callbacks=[early_stopping,check_point]
    
)

Epoch 1/6
178/178 [==============================] - ETA: 0s - loss: 0.1869 - accuracy: 0.9394
Epoch 1: val_accuracy improved from -inf to 0.95469, saving model to best_model.s2_v2


INFO:tensorflow:Assets written to: best_model.s2_v2\assets


INFO:tensorflow:Assets written to: best_model.s2_v2\assets


178/178 [==============================] - 655s 4s/step - loss: 0.1869 - accuracy: 0.9394 - val_loss: 0.1683 - val_accuracy: 0.9547
Epoch 2/6
178/178 [==============================] - ETA: 0s - loss: 0.1678 - accuracy: 0.9542
Epoch 2: val_accuracy did not improve from 0.95469
178/178 [==============================] - 1775s 10s/step - loss: 0.1678 - accuracy: 0.9542 - val_loss: 0.5122 - val_accuracy: 0.8648
Epoch 3/6
126/178 [====================>.........] - ETA: 2:42 - loss: 0.1232 - accuracy: 0.9636

KeyboardInterrupt: 

In [33]:
loss, accuracy = load_model.evaluate(val_s2_generator)
print(loss, accuracy)

41/41 [==============================] - 27s 651ms/step - loss: 0.2095 - accuracy: 0.9527
0.2095448225736618 0.952707827091217
